In [41]:
import asyncio
import pandas as pd
from playwright.async_api import async_playwright
import json
import sqlite3


In [42]:
async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        try:
            await page.goto("https://www.sportsbookreview.com/betting-odds/nba-basketball/?date=2021-3-05", wait_until="domcontentloaded")
            locator = page.locator("#__NEXT_DATA__")
            raw_json = await locator.inner_text()
            data = json.loads(raw_json)
        except Exception as e:
            print(f"Error: {e}")
        finally:
            await browser.close()
    return data
data = await main()

In [43]:
dframe = pd.DataFrame(columns=["away_TEAM", "home_TEAM","homeMoneyLine", "awayMoneyLine", "homeSpread","awaySpread"])

In [44]:
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()
dates_20_21 = pd.read_sql_query("SELECT [GAME_DATE] FROM [2020-2021 Team Data Combined]", conn)

In [45]:
dates_20_21 = dates_20_21['GAME_DATE'].unique()
dates_20_21 = dates_20_21.tolist()

In [46]:
dates_20_21 = [x[0:10] for x in dates_20_21]

In [47]:
for index, x in enumerate((data['props']['pageProps']['oddsTables'][0]['oddsTableModel']['gameRows'])):
    homeMoneyLine, awayMoneyLine, homeSpread, awaySpread = 0,0,0,0
    for z in x["oddsViews"]:
        homeMoneyLine += z['currentLine']["homeOdds"]
        awayMoneyLine += z['currentLine']["awayOdds"]
        homeSpread += z['currentLine']['homeSpread']
        awaySpread += z['currentLine']['awaySpread']
    homeMoneyLine = round(homeMoneyLine/len(x['oddsViews']),2)
    awayMoneyLine = round(awayMoneyLine/len(x['oddsViews']),2)
    homeSpread = round(homeSpread/len(x['oddsViews']),2)
    awaySpread = round(awaySpread/len(x['oddsViews']),2)
    dframe.loc[index] = ({"away_TEAM":x['gameView']['awayTeam']['fullName'],
    "home_TEAM":x['gameView']['homeTeam']['fullName'],
    "homeMoneyLine":homeMoneyLine,
    "awayMoneyLine":awayMoneyLine,
    "homeSpread":homeSpread,
    "awaySpread":awaySpread
    }
    )
dframe

IndexError: list index out of range

0